In [1]:
import pandas as pd 
import numpy as np 
from bs4 import BeautifulSoup 
import requests

### In this part of the problem, I am retrieving the table from the Wikipedia url and cleaning up the data for further use.

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [3]:
dfs = pd.read_html(url)

In [4]:
df = dfs[0]

In [5]:
df

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


In [6]:
df = df[df['Borough']!='Not assigned']

In [7]:
df['Neighbourhood'].replace('Not assigned', df['Borough'], inplace=True)

C:\Users\ariel\Anaconda3\lib\site-packages\pandas\core\generic.py:6746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [8]:
df.shape

(103, 3)

### In the next part of the problem, I import geocoder and the necessary code to get the geographical coordinates. Then I add two additional columns to our dataframe for the coordinates.

In [9]:
df2 = pd.read_csv('C:/Users/ariel/Downloads/Geospatial_Coordinates.csv')

In [10]:
df2.head(10)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


### Here, I sort the values of both dataframes so that when I add the coordinate columns, it will be lined up. 

In [11]:
df.sort_values(by='Postal Code', ascending=True, inplace=True)
df2.sort_values(by='Postal Code', ascending=True, inplace=True)

C:\Users\ariel\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [12]:
df = df.reset_index()

In [13]:
df.drop(columns='index', inplace=True)

In [14]:
df['Latitude'] = df2['Latitude']
df['Longitude'] = df2['Longitude']

In [15]:
df.tail(10)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
93,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
94,M9B,Etobicoke,"West Deane Park, Princess Gardens, Martin Grov...",43.650943,-79.554724
95,M9C,Etobicoke,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",43.643515,-79.577201
96,M9L,North York,Humber Summit,43.756303,-79.565963
97,M9M,North York,"Humberlea, Emery",43.724766,-79.532242
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437
102,M9W,Etobicoke,"Northwest, West Humber - Clairville",43.706748,-79.594054


In [16]:
from geopy.geocoders import Nominatim
import matplotlib.cm as cm 
import matplotlib.colors as colors
from sklearn.cluster import KMeans 
import folium

### Here, we defined the address we will need in order to define an instance of geocoder. Now we have the coordinates of Toronto.

In [17]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent='on_explorer')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geographical coordinates of Toronto are 43.6534817, -79.3839347.


### This code will create a map of Toronto with neighborhoods superimposed on top. 

In [18]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'],df['Neighbourhood']):
    label = '{}. {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat, lng],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_toronto)
    
map_toronto

### Now we'll focus on a specific borough, say Scarborough.

In [19]:
scarborough_data = df[df['Borough'] == 'Scarborough'].reset_index(drop=True)
scarborough_data.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [20]:
address = 'Scarborough, ON'

geolocatior = Nominatim(user_agent='on_explorer')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinates of Scarborough are {}, {}.'.format(latitude, longitude))

The geographical coordinates of Scarborough are 43.773077, -79.257774.


In [21]:
map_scarborough = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, lng, label in zip(scarborough_data['Latitude'], scarborough_data['Longitude'], scarborough_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat, lng],
    radius=5,
    popup=label,
    color='blue',
    fill=True, 
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_scarborough)
    
map_scarborough

### It's time to use the Foursquare API and explore the neighborhoods as well as segment them.

In [22]:
CLIENT_ID = 'BYMMXPKNXTC1UOJZTL5EQ0SF1EZCB3E41VVQIT3Y53TEWIET' # your Foursquare ID
CLIENT_SECRET = '4TSS5HBWU30PFKTNUGUXSD2UFMYJU1LJQC41LA2LSDW4G5QG' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: BYMMXPKNXTC1UOJZTL5EQ0SF1EZCB3E41VVQIT3Y53TEWIET
CLIENT_SECRET:4TSS5HBWU30PFKTNUGUXSD2UFMYJU1LJQC41LA2LSDW4G5QG


### Let's get some information on one of the neighbourhoods.

In [23]:
scarborough_data.loc[0, 'Neighbourhood']

'Malvern, Rouge'

In [24]:
neighbourhood_latitude = scarborough_data.loc[0, 'Latitude']
neighbourhood_longitude = scarborough_data.loc[0, 'Longitude']

neighbourhood_name = scarborough_data.loc[0, 'Neighbourhood']

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))

Latitude and longitude values of Malvern, Rouge are 43.806686299999996, -79.19435340000001.


In [25]:
LIMIT = 500 
radius = 500 

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighbourhood_latitude, 
    neighbourhood_longitude, 
    radius, 
    LIMIT)

url

'https://api.foursquare.com/v2/venues/explore?&client_id=BYMMXPKNXTC1UOJZTL5EQ0SF1EZCB3E41VVQIT3Y53TEWIET&client_secret=4TSS5HBWU30PFKTNUGUXSD2UFMYJU1LJQC41LA2LSDW4G5QG&v=20180605&ll=43.806686299999996,-79.19435340000001&radius=500&limit=500'

In [26]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f29d2a776ab466e80b45e17'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Malvern',
  'headerFullLocation': 'Malvern, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 1,
  'suggestedBounds': {'ne': {'lat': 43.8111863045, 'lng': -79.18812958073042},
   'sw': {'lat': 43.80218629549999, 'lng': -79.2005772192696}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bb6b9446edc76b0d771311c',
       'name': 'Wendy’s',
       'location': {'crossStreet': 'Morningside & Sheppard',
        'lat': 43.80744841934756,
        'lng': -79.19905558052072,
        'labeledLatLngs': [{'label': 'display'

### We'll need to extract the category of the venue so for that we build a function. 

In [27]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
    
    if len(categories_list) ==0:
        return None
    else:
        return categories_list[0]['name']

In [28]:
from pandas.io.json import json_normalize

In [29]:
venues = results['response']['groups'][0]['items']

nearby_venues = json_normalize(venues) 

filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]

nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

C:\Users\ariel\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Wendy’s,Fast Food Restaurant,43.807448,-79.199056


### There is only a Wendy's nearby. 

In [30]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

1 venues were returned by Foursquare.


### Let's make a function so that we can repeat the process for all neighborhoods in Scarborough 

In [36]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    for name, lat, lng, in zip(names, latitudes, longitudes):
        print(name) 
    
        #create the API request url 
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
    
        results = requests.get(url).json()['response']['groups'][0]['items']
    
        #return only relevant information for each nearby venue 
        venues_list.append([(
        name,
        lat,
        lng,
        v['venue']['name'],
        v['venue']['location']['lat'],
        v['venue']['location']['lng'],
        v['venue']['categories'][0]['name']) for v in results])
    
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood',
                            'Neighbourhood Latitude',
                             'Neighbourhood Longitude',
                             'Venue', 
                             'Venue Latitude', 
                             'Venue Longitude', 
                             'Venue Category']

    return(nearby_venues)

In [34]:
scarborough_data

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [37]:
scarborough_venues = getNearbyVenues(names=scarborough_data['Neighbourhood'],
                                    latitudes=scarborough_data['Latitude'],
                                    longitudes=scarborough_data['Longitude'])

Malvern, Rouge
Rouge Hill, Port Union, Highland Creek
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park, Ionview, East Birchmount Park
Golden Mile, Clairlea, Oakridge
Cliffside, Cliffcrest, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Wexford Heights, Scarborough Town Centre
Wexford, Maryvale
Agincourt
Clarks Corners, Tam O'Shanter, Sullivan
Milliken, Agincourt North, Steeles East, L'Amoreaux East
Steeles West, L'Amoreaux West
Upper Rouge


### Let's check the size of the new dataframe

In [38]:
print(scarborough_venues.shape)
scarborough_venues.head()

(96, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern, Rouge",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,RIGHT WAY TO GOLF,43.785177,-79.161108,Golf Course
2,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Great Shine Window Cleaning,43.783145,-79.157431,Home Service
3,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank


#### Let's check how many venues were returned for each neighborhood

In [39]:
scarborough_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Agincourt,5,5,5,5,5,5
"Birch Cliff, Cliffside West",4,4,4,4,4,4
Cedarbrae,9,9,9,9,9,9
"Clarks Corners, Tam O'Shanter, Sullivan",13,13,13,13,13,13
"Cliffside, Cliffcrest, Scarborough Village West",3,3,3,3,3,3
"Dorset Park, Wexford Heights, Scarborough Town Centre",6,6,6,6,6,6
"Golden Mile, Clairlea, Oakridge",10,10,10,10,10,10
"Guildwood, Morningside, West Hill",8,8,8,8,8,8
"Kennedy Park, Ionview, East Birchmount Park",7,7,7,7,7,7


#### Let's check how many unique categories there are in the returned venues

In [40]:
print('There are {} uniques categories.'.format(len(scarborough_venues['Venue Category'].unique())))

There are 55 uniques categories.


# Analyzing Each Neighborhood

In [42]:
#one hot encoding 
scarborough_onehot = pd.get_dummies(scarborough_venues[['Venue Category']], prefix="", prefix_sep="")

#add neighborhood column back to dataframe
scarborough_onehot['Neighbourhood'] = scarborough_venues['Neighbourhood']

#move neighborhood column to the first column 
fixed_columns = [scarborough_onehot.columns[-1]] + list(scarborough_onehot.columns[:-1])
scarborough_onehot = scarborough_onehot[fixed_columns]

scarborough_onehot.head()

,Neighbourhood,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bubble Tea Shop,Bus Line,...,Pharmacy,Pizza Place,Playground,Rental Car Location,Restaurant,Sandwich Place,Skating Rink,Soccer Field,Thai Restaurant,Vietnamese Restaurant
0,"Malvern, Rouge",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Rouge Hill, Port Union, Highland Creek",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Rouge Hill, Port Union, Highland Creek",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Rouge Hill, Port Union, Highland Creek",0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Guildwood, Morningside, West Hill",0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Check the new dataframe size

In [43]:
scarborough_onehot.shape

(96, 56)

#### Let's group the rows by neighborhood and take the mean of the frequency of occurence of each category 

In [44]:
scarborough_grouped = scarborough_onehot.groupby('Neighbourhood').mean().reset_index()
scarborough_grouped 

,Neighbourhood,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bubble Tea Shop,Bus Line,...,Pharmacy,Pizza Place,Playground,Rental Car Location,Restaurant,Sandwich Place,Skating Rink,Soccer Field,Thai Restaurant,Vietnamese Restaurant
0,Agincourt,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.0,...,0.000000,0.000000,0.0,0.000,0.000,0.000000,0.20,0.0,0.000000,0.000000
1,"Birch Cliff, Cliffside West",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.0,0.000,0.000,0.000000,0.25,0.0,0.000000,0.000000
2,Cedarbrae,0.000000,0.111111,0.000000,0.111111,0.111111,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.0,0.000,0.000,0.000000,0.00,0.0,0.111111,0.000000
3,"Clarks Corners, Tam O'Shanter, Sullivan",0.000000,0.000000,0.000000,0.000000,0.076923,0.000000,0.000000,0.000000,0.0,...,0.076923,0.153846,0.0,0.000,0.000,0.000000,0.00,0.0,0.076923,0.000000
4,"Cliffside, Cliffcrest, Scarborough Village West",0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.0,0.000,0.000,0.000000,0.00,0.0,0.000000,0.000000
5,"Dorset Park, Wexford Heights, Scarborough Town...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.0,0.000,0.000,0.000000,0.00,0.0,0.000000,0.166667
6,"Golden Mile, Clairlea, Oakridge",0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.2,...,0.000000,0.000000,0.0,0.000,0.000,0.000000,0.00,0.1,0.000000,0.000000
7,"Guildwood, Morningside, West Hill",0.000000,0.000000,0.000000,0.000000,0.125000,0.000000,0.125000,0.000000,0.0,...,0.000000,0.000000,0.0,0.125,0.125,0.000000,0.00,0.0,0.000000,0.000000
8,"Kennedy Park, Ionview, East Birchmount Park",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.0,0.000,0.000,0.000000,0.00,0.0,0.000000,0.000000
9,"Malvern, Rouge",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.0,0.000,0.000,0.000000,0.00,0.0,0.000000,0.000000


#### Check new size

In [45]:
scarborough_grouped.shape

(16, 56)

#### Print each neighborhood along with the top 5 most common venues

In [46]:
num_top_venues = 5 

for hood in scarborough_grouped['Neighbourhood']:
    print('----'+hood+'----')
    temp = scarborough_grouped[scarborough_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue', 'freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq' : 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0               Skating Rink   0.2
1  Latin American Restaurant   0.2
2             Breakfast Spot   0.2
3                     Lounge   0.2
4             Clothing Store   0.2


----Birch Cliff, Cliffside West----
                   venue  freq
0           Skating Rink  0.25
1  General Entertainment  0.25
2                   Café  0.25
3        College Stadium  0.25
4    American Restaurant  0.00


----Cedarbrae----
                  venue  freq
0      Hakka Restaurant  0.11
1  Caribbean Restaurant  0.11
2       Thai Restaurant  0.11
3                Bakery  0.11
4                  Bank  0.11


----Clarks Corners, Tam O'Shanter, Sullivan----
                  venue  freq
0           Pizza Place  0.15
1          Noodle House  0.08
2           Gas Station  0.08
3   Fried Chicken Joint  0.08
4  Fast Food Restaurant  0.08


----Cliffside, Cliffcrest, Scarborough Village West----
                 venue  freq
0  American Restaurant  0.33
1 

#### Let's put that into a pandas dataframe 

In [47]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### Create the new dataframe and display the top 10 venues for each neighborhood. 

In [49]:
num_top_venues = 10 

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe 
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = scarborough_grouped['Neighbourhood']

for ind in np.arange(scarborough_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(scarborough_grouped.iloc[ind, :], num_top_venues)
    
neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Skating Rink,Breakfast Spot,Latin American Restaurant,Lounge,Clothing Store,Vietnamese Restaurant,Coffee Shop,General Entertainment,Gas Station,Fried Chicken Joint
1,"Birch Cliff, Cliffside West",Skating Rink,General Entertainment,Café,College Stadium,Vietnamese Restaurant,Clothing Store,Golf Course,Gas Station,Fried Chicken Joint,Fast Food Restaurant
2,Cedarbrae,Hakka Restaurant,Athletics & Sports,Thai Restaurant,Bakery,Bank,Gas Station,Fried Chicken Joint,Caribbean Restaurant,Lounge,Convenience Store
3,"Clarks Corners, Tam O'Shanter, Sullivan",Pizza Place,Intersection,Convenience Store,Noodle House,Italian Restaurant,Pharmacy,Gas Station,Bank,Fast Food Restaurant,Thai Restaurant
4,"Cliffside, Cliffcrest, Scarborough Village West",American Restaurant,Intersection,Motel,Auto Garage,Coffee Shop,General Entertainment,Gas Station,Athletics & Sports,Fried Chicken Joint,Fast Food Restaurant


## Cluster Neighborhoods 
### Run k-means to cluster the neighborhood into 5 clusters

In [51]:
kclusters = 5 

scarborough_grouped_clustering = scarborough_grouped.drop('Neighbourhood', 1)

#run k-means clustering 
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(scarborough_grouped_clustering)

#check cluster labels generated for each row in the dataframe 
kmeans.labels_[0:10]

array([1, 1, 1, 1, 0, 1, 1, 1, 1, 2])

#### Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood

In [52]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

scarborough_merged = scarborough_data 

scarborough_merged = scarborough_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

scarborough_merged.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,2.0,Fast Food Restaurant,Vietnamese Restaurant,Clothing Store,Golf Course,General Entertainment,Gas Station,Fried Chicken Joint,Electronics Store,Discount Store,Department Store
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,1.0,Home Service,Golf Course,Bar,Clothing Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,1.0,Electronics Store,Rental Car Location,Intersection,Breakfast Spot,Medical Center,Bank,Mexican Restaurant,Restaurant,Fast Food Restaurant,Fried Chicken Joint
3,M1G,Scarborough,Woburn,43.770992,-79.216917,4.0,Coffee Shop,Korean Restaurant,Vietnamese Restaurant,Hakka Restaurant,Golf Course,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,1.0,Hakka Restaurant,Athletics & Sports,Thai Restaurant,Bakery,Bank,Gas Station,Fried Chicken Joint,Caribbean Restaurant,Lounge,Convenience Store


#### Let's visualize the resulting clusters 

In [68]:
#create map 
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters 
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map 
markers_colors = [] 
for lat, lon, poi, cluster in zip(scarborough_merged['Latitude'], scarborough_merged['Longitude'], scarborough_merged['Neighbourhood'], scarborough_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
    [lat, lon],
    radius=5,
    popup=label,
    color=rainbow[1],
    fill =True,
    fill_color=rainbow[1],
    fill_opacity=0.7).add_to(map_clusters)

map_clusters

### Examining Clusters
#### Determining the discriminating venue categories that distinguish each cluster 

In [69]:
scarborough_merged.loc[scarborough_merged['Cluster Labels']==0, scarborough_merged.columns[[1]+list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Scarborough,0.0,American Restaurant,Intersection,Motel,Auto Garage,Coffee Shop,General Entertainment,Gas Station,Athletics & Sports,Fried Chicken Joint,Fast Food Restaurant


In [70]:
scarborough_merged.loc[scarborough_merged['Cluster Labels']==1, scarborough_merged.columns[[1]+list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Scarborough,1.0,Home Service,Golf Course,Bar,Clothing Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
2,Scarborough,1.0,Electronics Store,Rental Car Location,Intersection,Breakfast Spot,Medical Center,Bank,Mexican Restaurant,Restaurant,Fast Food Restaurant,Fried Chicken Joint
4,Scarborough,1.0,Hakka Restaurant,Athletics & Sports,Thai Restaurant,Bakery,Bank,Gas Station,Fried Chicken Joint,Caribbean Restaurant,Lounge,Convenience Store
6,Scarborough,1.0,Discount Store,Chinese Restaurant,Department Store,Bus Station,Convenience Store,Coffee Shop,Clothing Store,Golf Course,General Entertainment,Gas Station
7,Scarborough,1.0,Bakery,Bus Line,Metro Station,Bus Station,Soccer Field,Ice Cream Shop,Intersection,Park,College Stadium,Convenience Store
9,Scarborough,1.0,Skating Rink,General Entertainment,Café,College Stadium,Vietnamese Restaurant,Clothing Store,Golf Course,Gas Station,Fried Chicken Joint,Fast Food Restaurant
10,Scarborough,1.0,Indian Restaurant,Vietnamese Restaurant,Pet Store,Light Rail Station,Chinese Restaurant,Bank,Bar,General Entertainment,Gas Station,Athletics & Sports
11,Scarborough,1.0,Vietnamese Restaurant,Auto Garage,Sandwich Place,Bakery,Breakfast Spot,Middle Eastern Restaurant,Coffee Shop,General Entertainment,Gas Station,Fried Chicken Joint
12,Scarborough,1.0,Skating Rink,Breakfast Spot,Latin American Restaurant,Lounge,Clothing Store,Vietnamese Restaurant,Coffee Shop,General Entertainment,Gas Station,Fried Chicken Joint
13,Scarborough,1.0,Pizza Place,Intersection,Convenience Store,Noodle House,Italian Restaurant,Pharmacy,Gas Station,Bank,Fast Food Restaurant,Thai Restaurant


In [71]:
scarborough_merged.loc[scarborough_merged['Cluster Labels']==2, scarborough_merged.columns[[1]+list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,2.0,Fast Food Restaurant,Vietnamese Restaurant,Clothing Store,Golf Course,General Entertainment,Gas Station,Fried Chicken Joint,Electronics Store,Discount Store,Department Store


In [72]:
scarborough_merged.loc[scarborough_merged['Cluster Labels']==3, scarborough_merged.columns[[1]+list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Scarborough,3.0,Playground,Vietnamese Restaurant,Clothing Store,Golf Course,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
14,Scarborough,3.0,Playground,Park,Vietnamese Restaurant,Chinese Restaurant,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store


In [73]:
scarborough_merged.loc[scarborough_merged['Cluster Labels']==4, scarborough_merged.columns[[1]+list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Scarborough,4.0,Coffee Shop,Korean Restaurant,Vietnamese Restaurant,Hakka Restaurant,Golf Course,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
